# Detector de Plagio

- Calcular similitud entre párrafos
- Calcular similitud entre oraciones
- Calcular similitud entre el documento a evaluar y:
    - Otros documentos
    - Texto obtenido de internet en base a palabras clave
    - Libros

In [1]:
from gensim.models import KeyedVectors
import numpy as np
from repository.csv_tools import get_documents
from models.naive_bayes_utils import NaiveBayes
from models.lda_utils import get_topic_keywords
from document import Document
from nltk import word_tokenize
import util.file_manager as fm

## Para calcular la distancia entre palabras se utilizará un modelo preentrenado de Word Embeddings

In [2]:
word_vectors = KeyedVectors.load('../keyed_vectors/complete.kv')

In [3]:
word_vectors.wmdistance('yo comprar cebollas', 'el adquirir zanahorias')

14.306833555617764

In [4]:
documents = get_documents('../data/dataset.csv')
scrapped = get_documents('../data/scrapped.csv')
books = get_documents('../data/books.csv')

# Evaluación de documentos potencialmente plagiados

- Deben ser del mismo tema
- Los párrafos u oraciones deben ser considerables

In [5]:
classifier = NaiveBayes()
classifier.load_model(path = '../data/')

In [6]:
def is_useful(topic, document):
    return document.topic == topic

In [7]:
def is_useful_text(lemmatized_string, original_string):
    return lemmatized_string not in questions and len(word_tokenize(lemmatized_string)) > 5 and '?' not in original_string and '¿' not in original_string

In [8]:
print(len(documents))
#doc_eval = documents.pop(15)
doc_eval = Document(path = '../test_dataset/Test Document $ TP 1 - Larga Cola - Campassi Rodrigo  (copy).docx')

305


In [9]:
topic, _ = classifier.topic(doc_eval.lemmatized_string)
print('Tópico detectado: {}'.format(topic))

Tópico detectado: La larga cola


In [10]:
keywords, p = get_topic_keywords(doc_eval.lemmatized_string)
print('Palabras clave encontradas: {}'.format(keywords))

Palabras clave encontradas: largo cola ofrecer venta regla cliente demanda nicho oferta minorista


In [11]:
filtered_documents = [document for document in documents if is_useful(topic, document)]

In [12]:
filtered_web_texts = [document for document in scrapped if is_useful(keywords, document)]

In [13]:
filtered_books = [book for book in books if is_useful(topic, book)]

### Filtado de preguntas y consignas frecuentes

In [30]:
questions = [
    'concepto conducta adaptativo complejas sistema ascendente sistema',
    'sistema emergente autoorganizativo sistema evolutivo caos',
    'desarrollar idea concepto conducta adaptativo complejas sistema ascendente sistema',
    'particular conducta dictiostellum implicancias',
    'enuncie idea componente basa dar ejemplo cada',
    'enunciar ley coase originalmente ahora implicancias',
    'poder caracterizar industrial decir rifkin invento cada infraestructura cada etapa',
    'lectura rifkin clase actual elegir apellido brevemente',
    'pc activo diferencia envolver usuario conducta tipo problema naturaleza interactivo pc distinguir individuo educados poder orientados manejo ser trabajador conocimiento analista conceptuales',
    'definir caracterizar experiencia diferencia producto servicio',
    'ejemplo distinto real si conocer invente experiencia',
    'mismo aproximadamente porcentaje fin respecto usuario',
    'relacionar ley pareto masa largo cola',
    'dar ejemplo internacional empresa producto servicio modelo largo cola',
    'dar ejemplo internacional empresa producto modelo largo cola',
]

### Si es un PDF, entonces comparo por oraciones. El PDF no es capaz de separarse en párrafos de manera correcta

In [31]:
def is_pdf(document):
    return fm.is_pdf(document.title)

## Detección del plagio

- Definir un Treshold:
    - A partir de qué distancia se considera plagio.
    - A mayor distancia, más plagio se detecta, con más posibilidad de detectar parafraseos.
    - A mayor distancia, también se detectan varios falsos positivos.

- Comparar con otros TP's
    - Si alguno es PDF se comparan oraciones, en otro caso párrafos

- Comparar con texto scrappeado de internet:
    - Solo se comparan párrafos, así sean PDF's o no.

- Comparar con libros consultados:
    - Se comparan solo oraciones, todos los libros vienen en formato de oraciones
    

In [42]:
treshold = 2.1

paragraphs = []
sentences = []
paragraph_count = len(doc_eval.preprocessed_paragraphs)
sentence_count = len(doc_eval.preprocessed_sentences)

for document in filtered_documents:
    if is_pdf(document) or is_pdf(doc_eval):
        for i, sentence in enumerate(doc_eval.preprocessed_sentences):
            for j, other_sentence in enumerate(document.preprocessed_sentences):
                if is_useful_text(sentence, doc_eval.sentences[i]) and is_useful_text(other_sentence, document.sentences[j]):
                    distance = word_vectors.wmdistance(sentence.split(), other_sentence.split())
                    if distance <= treshold:
                        sentences.append(
                            (document, i, j, distance)
                        )
    else:
        for i, paragraph in enumerate(doc_eval.preprocessed_paragraphs):
            for j, other_paragraph in enumerate(document.preprocessed_paragraphs):
                if is_useful_text(paragraph,doc_eval.paragraphs[i]) and is_useful_text(other_paragraph, document.paragraphs[j]):
                    distance = word_vectors.wmdistance(paragraph, other_paragraph) 
                    if distance <= treshold:
                        paragraphs.append(
                            (document, i, j, distance)
                        )


for scrapped in filtered_web_texts:
    for i, paragraph in enumerate(scrapped.preprocessed_paragraphs):
        for j, other_paragraph in enumerate(doc_eval.preprocessed_paragraphs):
           if is_useful_text(other_paragraph, doc_eval.paragraphs[j]):
                distance = word_vectors.wmdistance(paragraph, other_paragraph)
                if distance <= treshold:
                    paragraphs.append(
                        (scrapped, j, i, distance)
                    )

    
for book in filtered_books:
    for i, sentence in enumerate(book.preprocessed_sentences):
        for j, other_sentence in enumerate(doc_eval.preprocessed_sentences):
            if is_useful_text(other_sentence, doc_eval.sentences[j]):
                distance = word_vectors.wmdistance(sentence, other_sentence)
                if distance <= treshold:
                    sentences.append(
                        (book, j, i, distance)
                    )


# Impresión de resultados

- Porcentaje de plagio
- Distancia entre oraciones y párrafos plagiados

In [44]:
from nltk import word_tokenize
paragraphs_percentage = (len(paragraphs) * 100) / paragraph_count
sentences_percentage = (len(sentences) * 100) / sentence_count

final_percentage = paragraphs_percentage + sentences_percentage

print('Porcentaje de plagio de párrafos: {}%'.format(round(paragraphs_percentage, 2)))
print('Porcentaje de plagio de oraciones: {}%'.format(round(sentences_percentage, 2)))
print('Porcentaje de plagio total: {}%'.format(round(final_percentage, 2)))
print('\n\n\n')

for document, index_doc_eval, index_document, distance in paragraphs:
    intersect = len(set(word_tokenize(doc_eval.preprocessed_paragraphs[index_doc_eval])).intersection(word_tokenize(document.preprocessed_paragraphs[index_document])))
    if distance != 0:
        print('Paragraph: \n{}\n\nOther Paragraph: \n\n{}\n\nDistance: {}\n\nIntersection: {}\n\n\n\n'.format(doc_eval.paragraphs[index_doc_eval], document.paragraphs[index_document], distance, intersect))

for document, index_doc_eval, index_document, distance in sentences:
    intersect = len(set(word_tokenize(doc_eval.preprocessed_sentences[index_doc_eval])).intersection(word_tokenize(document.preprocessed_sentences[index_document])))
    if distance != 0:
        print('Sentence: \n{}\n\nOther sentence: \n\n{}\n\nDistance: {}\n\nIntersection: {}\n\n\n\n'.format(doc_eval.preprocessed_sentences[index_doc_eval], document.sentences[index_document], distance, intersect))

aumenta el boca en boca que se da personalmente o vía internet entre los diferentes consumidores. Esto, junto a las sugerencias de los usuarios y las reseñas de productos, produce que los productos ofrecidos sean encontrados por los potenciales demandantes.

Other Paragraph: 

 Respuestas: 1) La Larga Cola fue un término propuesto por Chris Anderson, el cual propone con este concepto que, gracias a las nuevas tecnologías, estamos pasando de una economía de masas a una economía de nichos. El objetivo es: el futuro de los negocios es vender menos cantidades de más productos. La Larga Cola no es otra cosa que un modelo de negocios que se fundamenta en vender productos poco comunes que gracias a Internet ven reducidos sus costos de almacenamiento y distribución. Se dice que es el presente de la economía minorista, ya que se trata de los productos que tienen poca venta pero que la suma de esos distintos productos (relacionados con los productos segmentados en nichos)

Distance: 1.8507574660